# High-level Keras (TF) Example

In [1]:
# Parameters
EPOCHS = 10
N_CLASSES=10
BATCHSIZE = 64
LR = 0.01
MOMENTUM = 0.9
GPU = True

LOGGER_URL='msdlvm.southcentralus.cloudapp.azure.com'
LOGGER_USRENAME='admin'
LOGGER_PASSWORD='password'
LOGGER_DB='gpudata'
LOGGER_SERIES='gpu'

In [7]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from os import path

from utils import cifar_for_library, yield_mb
from nb_logging import NotebookLogger, output_to, error_to
from gpumon import db_log_context
import codecs

from influxdb import InfluxDBClient

In [ ]:
client = InfluxDBClient(LOGGER_URL, 8086, LOGGER_USRENAME, LOGGER_PASSWORD, LOGGER_DB)

In [ ]:
node_id = os.getenv('AZ_BATCH_NODE_ID', default='node')
task_id = os.getenv('AZ_BATCH_TASK_ID', default='keras_tf')
job_id = os.getenv('AZ_BATCH_JOB_ID', default='keras_tf')

In [ ]:
logger = create_logger(client, node_id=node_id, task_id=task_id, job_id=job_id)

In [3]:
nb_teminal_logger = NotebookLogger(sys.stdout.session, sys.stdout.pub_thread, sys.stdout.name, sys.__stdout__)

In [4]:
rst_out = output_to(nb_teminal_logger)
rst_err = error_to(nb_teminal_logger)

In [5]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
# Channels should be first (otherwise slow)
print(K.backend.image_data_format())

OS:  linux
Python:  3.5.4 |Anaconda, Inc.| (default, Oct  5 2017, 08:00:22) 
[GCC 7.2.0]
Keras:  2.0.8
Numpy:  1.13.3
Tensorflow:  1.3.0
tensorflow
channels_last


In [8]:
data_path = path.join(os.getenv('AZ_BATCHAI_INPUT_DATASET'), 'cifar-10-batches-py')

In [9]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [10]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [16]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(data_path, channel_first=False, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Done.
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 748 ms, sys: 644 ms, total: 1.39 s
Wall time: 1.39 s


In [17]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 16 ms, sys: 76 ms, total: 92 ms
Wall time: 89.1 ms


In [18]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 4 ms, sys: 20 ms, total: 24 ms
Wall time: 23.1 ms


In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 50)        22550     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 100)       45100     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 100)       90100     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 100)         0         
__________

In [20]:
with Timer() as t:
    with db_log_context(LOGGER_URL, '8086', LOGGER_USRENAME, LOGGER_PASSWORD, LOGGER_DB, LOGGER_SERIES, 
                    node_id=node_id, task_id=task_id, job_id=job_id):
        # Train model
        model.fit(x_train,
                  y_train,
                  batch_size=BATCHSIZE,
                  epochs=EPOCHS,
                  verbose=1)
print('Training took %.03f sec.' % t.interval)
logger('log', training_duration=t.interval)

Epoch 1/10
50000/50000 [==============================] - 27s - loss: 1.8429 - acc: 0.3278    
Epoch 2/10
50000/50000 [==============================] - 22s - loss: 1.4025 - acc: 0.4924    - 
Epoch 3/10
50000/50000 [==============================] - 21s - loss: 1.1723 - acc: 0.5803    
Epoch 4/10
50000/50000 [==============================] - 20s - loss: 1.0221 - acc: 0.6359    
Epoch 5/10
50000/50000 [==============================] - 21s - loss: 0.9057 - acc: 0.6801    
Epoch 6/10
50000/50000 [==============================] - 20s - loss: 0.8209 - acc: 0.7095    
Epoch 7/10
50000/50000 [==============================] - 21s - loss: 0.7459 - acc: 0.7353    
Epoch 8/10
50000/50000 [==============================] - 20s - loss: 0.6832 - acc: 0.7589    
Epoch 9/10
50000/50000 [==============================] - 21s - loss: 0.6219 - acc: 0.7811    - ETA: 1s - loss:
Epoch 10/10
50000/50000 [==============================] - 20s - loss: 0.5710 - acc: 0.7991    
CPU times: user 1min 39s, sys:

In [21]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 864 ms, sys: 608 ms, total: 1.47 s
Wall time: 1.82 s


In [22]:
acc=sum(y_guess == y_truth)/len(y_guess)
print("Accuracy: ", acc)
logger('log', test_accuracy=acc)

Accuracy:  0.7728
